In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-storm-4/Data-dictionary.xlsx
/kaggle/input/data-storm-4/DataStorm4.0_KaggleProblemStatement.pdf
/kaggle/input/data-storm-4/Store-info.csv
/kaggle/input/data-storm-4/Testing-data.csv
/kaggle/input/data-storm-4/Historical-transaction-data.csv


In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df_1 = pd.read_csv('/kaggle/input/data-storm-4/Historical-transaction-data.csv')
df_2 = pd.read_csv('/kaggle/input/data-storm-4/Store-info.csv')
df_3=pd.read_csv("/kaggle/input/data-storm-4/Testing-data.csv")

In [4]:
null_counts = df_1.isnull().sum()

# Print the null value counts for each column
print(null_counts)

item_description    35928
transaction_date        0
invoice_id           6320
customer_id             0
shop_id                 0
item_price              0
quantity_sold           0
dtype: int64


In [5]:
df_1 = df_1.drop_duplicates()

In [6]:
df_1['transaction_date'].isnull().sum()

0

In [7]:
dates = df_1['transaction_date']
Dates = []

# Extract the substring corresponding to the day component
for date in dates:
    day_string = date[8:10]
    
    # Convert the day string to an integer
    day = int(day_string)
    
    Dates.append(day)
    
    
df_1['date'] =  Dates

In [8]:
df_1['date'].isnull().sum()

0

In [9]:
sl_holidays = [pd.to_datetime(d) for d in ['2021-10-19', '2021-10-20', '2021-11-04', '2022-11-18', 
                                           '2022-12-18', '2022-12-25']]

# assume that the transaction_date column is a string column
df_1['transaction_date'] = pd.to_datetime(df_1['transaction_date'])

# now you can use the .dt accessor on the transaction_date column
df_1['is_weekend'] = df_1['transaction_date'].dt.weekday.isin([5, 6])

df_1['is_holiday'] = df_1['transaction_date'].isin(sl_holidays)

# convert boolean values to integers
df_1['is_weekend'] = df_1['is_weekend'].astype(int)

df_1['is_holiday'] = df_1['is_holiday'].astype(int)

In [10]:
df_1.head()

,item_description,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold,date,is_weekend,is_holiday
0,ORANGE BARLEY 1.5L,2021-12-11 00:00:00+00:00,147.0,BGXA,SHOP008,220,2,11,1,0
1,GINGER BEER 1.5L,2021-10-17 00:00:00+00:00,371.0,IA25,SHOP112,220,2,17,1,0
2,TONIC PET 500ML,2021-12-13 00:00:00+00:00,484.0,VN7V,SHOP008,160,2,13,0,0
3,CREAM SODA 1L,2021-12-13 00:00:00+00:00,484.0,VN7V,SHOP008,150,2,13,0,0
4,STRAWBERRY MILK 180ML,2021-10-23 00:00:00+00:00,1310.0,7S00,SHOP112,210,5,23,1,0


In [11]:

# extract item_name and item_size from item_description
df_1[['item_name', 'item_size']] = df_1['item_description'].str.extract('([\w\s]+) (\d*\.?\d+)\w{0,2}$')

# convert item_size to liters
df_1['item_size'] = df_1['item_size'].astype(float)
df_1.loc[df_1['item_description'].str.contains('ML', na=False), 'item_size'] /= 1000

# display the result
df_1.head(5)

,item_description,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold,date,is_weekend,is_holiday,item_name,item_size
0,ORANGE BARLEY 1.5L,2021-12-11 00:00:00+00:00,147.0,BGXA,SHOP008,220,2,11,1,0,ORANGE BARLEY,1.50
1,GINGER BEER 1.5L,2021-10-17 00:00:00+00:00,371.0,IA25,SHOP112,220,2,17,1,0,GINGER BEER,1.50
2,TONIC PET 500ML,2021-12-13 00:00:00+00:00,484.0,VN7V,SHOP008,160,2,13,0,0,TONIC PET,0.50
3,CREAM SODA 1L,2021-12-13 00:00:00+00:00,484.0,VN7V,SHOP008,150,2,13,0,0,CREAM SODA,1.00
4,STRAWBERRY MILK 180ML,2021-10-23 00:00:00+00:00,1310.0,7S00,SHOP112,210,5,23,1,0,STRAWBERRY MILK,0.18


In [12]:
# Merge the two DataFrames on the primary key column
df_1 = pd.merge(df_1, df_2, on='shop_id')


In [13]:
new_order = ['transaction_date','invoice_id','customer_id','shop_id','item_name','item_size','item_price','quantity_sold', 'shop_area_sq_ft','date','is_weekend','is_holiday', 'shop_profile']

# Reorder the columns in the dataframe
df_1 = df_1[new_order]
df_1.head()

,transaction_date,invoice_id,customer_id,shop_id,item_name,item_size,item_price,quantity_sold,shop_area_sq_ft,date,is_weekend,is_holiday,shop_profile
0,2021-12-11 00:00:00+00:00,147.0,BGXA,SHOP008,ORANGE BARLEY,1.5,220,2,678,11,1,0,Moderate
1,2021-12-13 00:00:00+00:00,484.0,VN7V,SHOP008,TONIC PET,0.5,160,2,678,13,0,0,Moderate
2,2021-12-13 00:00:00+00:00,484.0,VN7V,SHOP008,CREAM SODA,1.0,150,2,678,13,0,0,Moderate
3,2021-12-10 00:00:00+00:00,1000053.0,VT9C,SHOP008,GINGER BEER,1.5,220,1,678,10,0,0,Moderate
4,2021-12-10 00:00:00+00:00,1000057.0,8QLS,SHOP008,GINGER BEER,1.5,440,1,678,10,0,0,Moderate


In [14]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Sample dataset
for i in df_1:
    if i not in ['transaction_date','invoice_id','customer_id','shop_id','item_name','shop_profile','item_encoded','is_weekend','is_holiday']:
        col_data = df_1[i]

    # Perform min-max scaling on the selected column
        scaled_data = (col_data - col_data.min()) / (col_data.max() - col_data.min())

        # Update the selected column in the original data frame with the scaled data
        df_1[i] = scaled_data

In [15]:
df_1 = df_1.dropna(subset=['item_name'])

In [16]:
df_1.head()

,transaction_date,invoice_id,customer_id,shop_id,item_name,item_size,item_price,quantity_sold,shop_area_sq_ft,date,is_weekend,is_holiday,shop_profile
0,2021-12-11 00:00:00+00:00,147.0,BGXA,SHOP008,ORANGE BARLEY,1.000000,0.010654,0.029412,0.487805,0.333333,1,0,Moderate
1,2021-12-13 00:00:00+00:00,484.0,VN7V,SHOP008,TONIC PET,0.242424,0.007198,0.029412,0.487805,0.400000,0,0,Moderate
2,2021-12-13 00:00:00+00:00,484.0,VN7V,SHOP008,CREAM SODA,0.621212,0.006623,0.029412,0.487805,0.400000,0,0,Moderate
3,2021-12-10 00:00:00+00:00,1000053.0,VT9C,SHOP008,GINGER BEER,1.000000,0.010654,0.019608,0.487805,0.300000,0,0,Moderate
4,2021-12-10 00:00:00+00:00,1000057.0,8QLS,SHOP008,GINGER BEER,1.000000,0.023323,0.019608,0.487805,0.300000,0,0,Moderate


In [17]:
df_sorted = df_1.sort_values(by='shop_id')
df_1=df_sorted
df_1.head()

,transaction_date,invoice_id,customer_id,shop_id,item_name,item_size,item_price,quantity_sold,shop_area_sq_ft,date,is_weekend,is_holiday,shop_profile
50020,2021-10-16 00:00:00+00:00,71340.0,9HZ9,SHOP001,ORANGE BARLEY,1.000000,0.010654,0.019608,0.362003,0.500000,1,0,Low
50823,2021-10-23 00:00:00+00:00,72216.0,7DEN,SHOP001,ORANGE BARLEY,1.000000,0.010654,0.029412,0.362003,0.733333,1,0,Low
50824,2021-10-31 00:00:00+00:00,73198.0,V7TV,SHOP001,FIT O MIXED FRUIT,0.015152,0.004895,0.039216,0.362003,1.000000,1,0,Low
50825,2021-11-15 00:00:00+00:00,74839.0,Z959,SHOP001,SODA,0.242424,0.002016,0.019608,0.362003,0.466667,0,0,Low
50826,2021-11-15 00:00:00+00:00,74839.0,Z959,SHOP001,TONIC PET,0.242424,0.007198,0.019608,0.362003,0.466667,0,0,Low


In [18]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# create a sample DataFrame

# create a LabelEncoder object
le = LabelEncoder()

# fit and transform the 'fruits' column
df_1['item_encoded'] = le.fit_transform(df_1['item_name'])
# df_1['shop_encoded'] = le.fit_transform(df_1['shop_id'])
encoded_df=df_1

In [19]:
encoded_df.head()

,transaction_date,invoice_id,customer_id,shop_id,item_name,item_size,item_price,quantity_sold,shop_area_sq_ft,date,is_weekend,is_holiday,shop_profile,item_encoded
50020,2021-10-16 00:00:00+00:00,71340.0,9HZ9,SHOP001,ORANGE BARLEY,1.000000,0.010654,0.019608,0.362003,0.500000,1,0,Low,14
50823,2021-10-23 00:00:00+00:00,72216.0,7DEN,SHOP001,ORANGE BARLEY,1.000000,0.010654,0.029412,0.362003,0.733333,1,0,Low,14
50824,2021-10-31 00:00:00+00:00,73198.0,V7TV,SHOP001,FIT O MIXED FRUIT,0.015152,0.004895,0.039216,0.362003,1.000000,1,0,Low,6
50825,2021-11-15 00:00:00+00:00,74839.0,Z959,SHOP001,SODA,0.242424,0.002016,0.019608,0.362003,0.466667,0,0,Low,16
50826,2021-11-15 00:00:00+00:00,74839.0,Z959,SHOP001,TONIC PET,0.242424,0.007198,0.019608,0.362003,0.466667,0,0,Low,19


In [20]:
filtered_df = encoded_df[encoded_df['shop_profile'].notnull()]
na_df=encoded_df[encoded_df['shop_profile'].isnull()]
# encoded_df.head()
# filtered_df["shop_profile"]
# na_df.head()

In [21]:
# import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the CSV file into a pandas DataFrame
# df = pd.read_csv('filename.csv')

# Create a LabelEncoder object
le = LabelEncoder()

# Fit the LabelEncoder on the column and transform the column
#filtered_df['column_name_encoded'] = le.fit_transform(filtered_df["shop_profile"])

# Save the updated DataFrame to a new CSV file
# filtered_df.to_csv('filename_encoded.csv', index=False)

# define the mapping between original and encoded values
mapping = {'Moderate': 1, 'Low': 0, 'High': 2}

# fit the encoder to the 'shopprofile' column using the mapping
le.fit([mapping[val] for val in filtered_df['shop_profile']])

# transform the 'shopprofile' column using the fitted encoder
filtered_df['shop_profile_encoded'] = le.transform([mapping[val] for val in filtered_df['shop_profile']])


filtered_df.head()

,transaction_date,invoice_id,customer_id,shop_id,item_name,item_size,item_price,quantity_sold,shop_area_sq_ft,date,is_weekend,is_holiday,shop_profile,item_encoded,shop_profile_encoded
50020,2021-10-16 00:00:00+00:00,71340.0,9HZ9,SHOP001,ORANGE BARLEY,1.000000,0.010654,0.019608,0.362003,0.500000,1,0,Low,14,0
50823,2021-10-23 00:00:00+00:00,72216.0,7DEN,SHOP001,ORANGE BARLEY,1.000000,0.010654,0.029412,0.362003,0.733333,1,0,Low,14,0
50824,2021-10-31 00:00:00+00:00,73198.0,V7TV,SHOP001,FIT O MIXED FRUIT,0.015152,0.004895,0.039216,0.362003,1.000000,1,0,Low,6,0
50825,2021-11-15 00:00:00+00:00,74839.0,Z959,SHOP001,SODA,0.242424,0.002016,0.019608,0.362003,0.466667,0,0,Low,16,0
50826,2021-11-15 00:00:00+00:00,74839.0,Z959,SHOP001,TONIC PET,0.242424,0.007198,0.019608,0.362003,0.466667,0,0,Low,19,0


In [22]:
# calculate the index for splitting the dataframe
split_idx = int(0.8 * len(filtered_df))
# split the dataframe into training and testing sets
train_df =filtered_df.iloc[:split_idx]
test_df = filtered_df.iloc[split_idx:]
train_df.head()

,transaction_date,invoice_id,customer_id,shop_id,item_name,item_size,item_price,quantity_sold,shop_area_sq_ft,date,is_weekend,is_holiday,shop_profile,item_encoded,shop_profile_encoded
50020,2021-10-16 00:00:00+00:00,71340.0,9HZ9,SHOP001,ORANGE BARLEY,1.000000,0.010654,0.019608,0.362003,0.500000,1,0,Low,14,0
50823,2021-10-23 00:00:00+00:00,72216.0,7DEN,SHOP001,ORANGE BARLEY,1.000000,0.010654,0.029412,0.362003,0.733333,1,0,Low,14,0
50824,2021-10-31 00:00:00+00:00,73198.0,V7TV,SHOP001,FIT O MIXED FRUIT,0.015152,0.004895,0.039216,0.362003,1.000000,1,0,Low,6,0
50825,2021-11-15 00:00:00+00:00,74839.0,Z959,SHOP001,SODA,0.242424,0.002016,0.019608,0.362003,0.466667,0,0,Low,16,0
50826,2021-11-15 00:00:00+00:00,74839.0,Z959,SHOP001,TONIC PET,0.242424,0.007198,0.019608,0.362003,0.466667,0,0,Low,19,0


In [23]:
X_train=train_df.drop(columns=['transaction_date','invoice_id','customer_id','shop_id','shop_profile','shop_profile_encoded','item_name'])
y_train=train_df["shop_profile_encoded"]
X_test=test_df.drop(columns=['transaction_date','invoice_id','customer_id','shop_id','shop_profile','shop_profile_encoded','item_name'])
y_test=test_df["shop_profile_encoded"]

In [24]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# load the data into a DataFrame
# data = pd.read_csv('data.csv')

# # split the data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(data[['feature1', 'feature2', 'feature3']], data['target'], test_size=0.2)

# create a logistic regression object and fit the training data
lr = LogisticRegression()
lr.fit(X_train, y_train)

# evaluate the performance of the model on the test set
score = lr.score(X_test, y_test)
print("Accuracy: {:.2f}%".format(score * 100))

Accuracy: 32.28%


In [25]:
from sklearn.ensemble import RandomForestClassifier

# Create a random forest classifier with 100 trees
rf = RandomForestClassifier(n_estimators=100, random_state=50)

# Train the random forest classifier on the training data
rf.fit(X_train, y_train)

# Predict the classes of the testing data
y_pred = rf.predict(X_test)

# Evaluate the accuracy of the model
accuracy = rf.score(X_test, y_test)*100
print('Accuracy of the  RandomForestClassifier is equal ' + str(round(accuracy, 2)) + ' %.')

Accuracy of the  RandomForestClassifier is equal 47.52 %.


In [26]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

# Load the dataset into a pandas dataframe
# df = pd.read_csv('dataset.csv')

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.3)

# Create a decision tree classifier object
clf = DecisionTreeClassifier()

# Fit the classifier to the training data
clf.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = clf.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")

Accuracy: 0.49518168287806125


In [27]:
test_df = na_df
X_train=filtered_df.drop(columns=['transaction_date','invoice_id','customer_id','shop_id','shop_profile','shop_profile_encoded','item_name'])
y_train=filtered_df["shop_profile_encoded"]
X_test=test_df.drop(columns=['transaction_date','invoice_id','customer_id','shop_id','shop_profile','item_name'])

In [28]:
test_df.head()

,transaction_date,invoice_id,customer_id,shop_id,item_name,item_size,item_price,quantity_sold,shop_area_sq_ft,date,is_weekend,is_holiday,shop_profile,item_encoded
430195,2021-11-28 00:00:00+00:00,982046.0,IQS6,SHOP002,CREAM SODA,1.000000,0.035992,0.049020,0.435173,0.900000,1,0,NaN,2
430203,2021-12-11 00:00:00+00:00,985207.0,FQ8B,SHOP002,CREAM SODA,0.242424,0.003743,0.019608,0.435173,0.333333,1,0,NaN,2
430193,2021-11-27 00:00:00+00:00,981889.0,5PKH,SHOP002,CREAM SODA,0.242424,0.003743,0.019608,0.435173,0.866667,1,0,NaN,2
430194,2021-11-27 00:00:00+00:00,981889.0,5PKH,SHOP002,CREAM SODA,0.242424,0.003743,0.029412,0.435173,0.866667,1,0,NaN,2
430196,2021-11-28 00:00:00+00:00,982172.0,YT2T,SHOP002,CREAM SODA,0.621212,0.006623,0.019608,0.435173,0.900000,1,0,NaN,2


In [29]:
print(test_df.shape[0])
print(X_test.shape[0])

75116
75116


In [30]:
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [31]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

# Load the dataset into a pandas dataframe
# df = pd.read_csv('dataset.csv')

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.3)

# Create a decision tree classifier object
clf = DecisionTreeClassifier()

# Fit the classifier to the training data
clf.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = clf.predict(X_test)

In [32]:
df_4 = pd.DataFrame({'y_pred': y_pred})
df_4

,y_pred
0,1
1,1
2,1
3,1
4,1
...,...
75111,0
75112,0
75113,0
75114,0


In [33]:
test_df = test_df.reset_index(drop=True)
test_df

,transaction_date,invoice_id,customer_id,shop_id,item_name,item_size,item_price,quantity_sold,shop_area_sq_ft,date,is_weekend,is_holiday,shop_profile,item_encoded
0,2021-11-28 00:00:00+00:00,982046.0,IQS6,SHOP002,CREAM SODA,1.000000,0.035992,0.049020,0.435173,0.900000,1,0,NaN,2
1,2021-12-11 00:00:00+00:00,985207.0,FQ8B,SHOP002,CREAM SODA,0.242424,0.003743,0.019608,0.435173,0.333333,1,0,NaN,2
2,2021-11-27 00:00:00+00:00,981889.0,5PKH,SHOP002,CREAM SODA,0.242424,0.003743,0.019608,0.435173,0.866667,1,0,NaN,2
3,2021-11-27 00:00:00+00:00,981889.0,5PKH,SHOP002,CREAM SODA,0.242424,0.003743,0.029412,0.435173,0.866667,1,0,NaN,2
4,2021-11-28 00:00:00+00:00,982172.0,YT2T,SHOP002,CREAM SODA,0.621212,0.006623,0.019608,0.435173,0.900000,1,0,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75111,2021-12-05 00:00:00+00:00,818431.0,P4BG,SHOP114,ORANGE CRUSH,0.242424,0.003743,0.019608,0.275995,0.133333,1,0,NaN,15
75112,2021-12-05 00:00:00+00:00,1000014.0,BILT,SHOP114,NECTO,1.000000,0.010654,0.029412,0.275995,0.133333,1,0,NaN,13
75113,2021-12-06 00:00:00+00:00,1000153.0,Y3N3,SHOP114,FIT O ORANGE,0.015152,0.002016,0.019608,0.275995,0.166667,0,0,NaN,7
75114,2021-10-15 00:00:00+00:00,5134724.0,8SH2,SHOP114,GINGER BEER,0.166667,0.003743,0.029412,0.275995,0.466667,0,0,NaN,8


In [34]:
result_df = pd.concat([test_df, df_4], axis=1)

In [35]:
new_df =result_df.loc[:, ["shop_id", "y_pred"]]
new_df

,shop_id,y_pred
0,SHOP002,1
1,SHOP002,1
2,SHOP002,1
3,SHOP002,1
4,SHOP002,1
...,...,...
75111,SHOP114,0
75112,SHOP114,0
75113,SHOP114,0
75114,SHOP114,0


In [36]:
mode_df = new_df.groupby('shop_id')['y_pred'].apply(lambda x: x.mean()).reset_index()

mode_df['y_pred'] = mode_df['y_pred'].astype(int)

In [37]:
mode_df=mode_df.replace([0,1,2],["Low","Moderate","High"])

In [38]:
mode_df = mode_df.rename(columns={'y_pred': 'shop_profile'})

In [39]:
df_3.head()

,shop_id,shop_profile
0,SHOP046,NaN
1,SHOP024,NaN
2,SHOP023,NaN
3,SHOP097,NaN
4,SHOP044,NaN


In [40]:
# df_3=pd.read_csv("/kaggle/input/data-storm-4/Testing-data.csv")
df_3.drop("shop_profile", axis =1, inplace = True)
merged_df = pd.merge(df_3, mode_df, on='shop_id')

In [41]:
merged_df.head()

,shop_id,shop_profile
0,SHOP046,Low
1,SHOP024,Low
2,SHOP023,Low
3,SHOP097,Moderate
4,SHOP044,Low


In [42]:
merged_df.to_csv("Day_2_Submission1.csv",index=False)